In [1]:
import pandas as pd
import os

In [2]:
file_paths = {
    "VDJdb_beta": vdjdb_beta_read_path,
    "McPAS_beta": mcpastcr_beta_read_path,
    "IEDB_beta": iedb_beta_read_path,
    "pMTnet_beta": pmtnet_beta_read_path,
    "VDJdb_paired": vdjdb_paired_read_path,
    "McPAS_paired": mcpastcr_paired_read_path,
    "IEDB_paired": iedb_paired_read_path
}

NameError: name 'vdjdb_beta_read_path' is not defined

In [ ]:
import pandas as pd
import os

# Function to classify tasks based on TCR and epitope presence
def calculate_task(row, known_epitopes, known_tcr, paired=False):
    if paired:
        tra_cdr3 = str(row['TRA_CDR3']) if pd.notna(row['TRA_CDR3']) else ''
        trb_cdr3 = str(row['TRB_CDR3']) if pd.notna(row['TRB_CDR3']) else ''
        tcr = tra_cdr3 + '_' + trb_cdr3
    else:
        tcr = row['TRB_CDR3']
    
    epitope_exists = row['Epitope'] in known_epitopes
    cdr3_exists = tcr in known_tcr
    
    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")

# Placeholder for the data
all_data = {}

# Load and prepare data
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # Create tcr_key
            if "TRA_CDR3" in df.columns:
                paired = True
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                paired = False
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df
            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Classify TPP tasks
for test_file_name, test_df in all_data.items():
    # Define training data (excluding the current test set)
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # Determine if it's a paired dataset
    paired = "TRA_CDR3" in test_df.columns

    # Apply classification
    test_df['task'] = test_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=paired), axis=1)

    # Count TPP3 pairs
    tpp3_pairs = (test_df['task'] == 'TPP3').sum()

    # Print the results
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")


In [ ]:
import pandas as pd
import os

# Function to classify TPP tasks based on TCR and epitope presence
def calculate_task(row, known_epitopes, known_tcr, paired=False):
    if paired:
        tra_cdr3 = str(row['TRA_CDR3']) if pd.notna(row['TRA_CDR3']) else ''
        trb_cdr3 = str(row['TRB_CDR3']) if pd.notna(row['TRB_CDR3']) else ''
        tcr = tra_cdr3 + '_' + trb_cdr3
    else:
        tcr = row['TRB_CDR3']
    
    epitope_exists = row['Epitope'] in known_epitopes
    cdr3_exists = tcr in known_tcr
    
    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")


# Load train and validation data
train_file = f'{pipeline_data_splitted}/{precision}/beta/train.tsv'
validation_file = f'{pipeline_data_splitted}/{precision}/beta/validation.tsv'
test_file = f'{pipeline_data_splitted}/{precision}/beta/test.tsv'
vdjdb_test_file = vdjdb_beta_read_path  # Path to the VDJdb test dataset

df_train = pd.read_csv(train_file, sep='\t')
df_validation = pd.read_csv(validation_file, sep='\t')
df_test = pd.read_csv(test_file, sep='\t')

# Combine train and validation datasets
trainval_df = pd.concat([df_train, df_validation], ignore_index=True)

# Load VDJdb test data
vdjdb_df = pd.read_csv(vdjdb_test_file, sep='\t') #f'{pipeline_data_cleaned}/VDJdb/VDJdb_cleaned_data_beta.tsv'

# Create tcr_key and clean data
trainval_df["tcr_key"] = trainval_df["TRB_CDR3"].astype(str).str.strip()
trainval_df["Epitope"] = trainval_df["Epitope"].astype(str).str.strip()

vdjdb_df["tcr_key"] = vdjdb_df["TRB_CDR3"].astype(str).str.strip()
vdjdb_df["Epitope"] = vdjdb_df["Epitope"].astype(str).str.strip()

df_test["tcr_key"] = df_test["TRB_CDR3"].astype(str).str.strip()
df_test["Epitope"] = df_test["Epitope"].astype(str).str.strip()

# Generate lookup sets for fast comparison
seen_tcrs = set(trainval_df["tcr_key"])
seen_epitopes = set(trainval_df["Epitope"])

# Apply classification to the VDJdb dataset
vdjdb_df['task'] = vdjdb_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=False), axis=1)

# Count TPP3 pairs
tpp3_pairs = (vdjdb_df['task'] == 'TPP3').sum()

# Print the results
print(f"\n**TPP Analysis for VDJdb with Train + Validation**")
print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
print(f"  - Gesamt Test-Paare: {len(vdjdb_df)}")


In [ ]:
# Original TPP3 Paare vor dem Hinzufügen negativer Daten
original_tpp3 = vdjdb_df[(vdjdb_df['task'] == 'TPP3')][["tcr_key", "Epitope"]]

# Prüfen, ob diese TPP3-Paare in den negativen Daten des Train/Validation-Sets auftauchen
negative_in_trainval = trainval_df[trainval_df['Binding'] == 0][["tcr_key", "Epitope"]]

# Vergleichen
tpp3_now_seen = original_tpp3.merge(negative_in_trainval, on=["tcr_key", "Epitope"], how="inner")

print(f"Anzahl der ursprünglichen TPP3-Paare, die jetzt in negativen Daten des Train/Validation-Sets vorkommen: {len(tpp3_now_seen)}")

# Zeige einige Beispiele
if not tpp3_now_seen.empty:
    print(tpp3_now_seen.head(10))
else:
    print("Keine der ursprünglichen TPP3-Paare wurden in den negativen Daten gefunden.")

# Prüfen, ob die ursprünglichen TPP3-Paare in den positiven Daten des Train/Validation-Sets enthalten sind
positive_in_trainval = trainval_df[trainval_df['Binding'] == 1][["tcr_key", "Epitope"]]

# Vergleich durchführen
tpp3_in_pos_trainval = original_tpp3.merge(positive_in_trainval, on=["tcr_key", "Epitope"], how="inner")

print(f"Anzahl der ursprünglichen TPP3-Paare, die jetzt in den positiven Train/Validation-Daten vorkommen: {len(tpp3_in_pos_trainval)}")

In [ ]:
# Paths to the concatenated files
concatenated_beta_file = f'{pipeline_data_concatenated}/{precision}/beta_concatenated.tsv'
vdjdb_test_file = vdjdb_beta_read_path  # Path to the original VDJdb data

# Load concatenated beta data
concatenated_beta_df = pd.read_csv(concatenated_beta_file, sep='\t')

# Load original VDJdb dataset
vdjdb_df = pd.read_csv(vdjdb_test_file, sep='\t')

# Create tcr_key and clean data
concatenated_beta_df["tcr_key"] = concatenated_beta_df["TRB_CDR3"].astype(str).str.strip()
concatenated_beta_df["Epitope"] = concatenated_beta_df["Epitope"].astype(str).str.strip()

vdjdb_df["tcr_key"] = vdjdb_df["TRB_CDR3"].astype(str).str.strip()
vdjdb_df["Epitope"] = vdjdb_df["Epitope"].astype(str).str.strip()

# Generate lookup sets from concatenated data
seen_tcrs = set(concatenated_beta_df["tcr_key"])
seen_epitopes = set(concatenated_beta_df["Epitope"])

# Apply classification to the VDJdb dataset
vdjdb_df['task'] = vdjdb_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=False), axis=1)

# Count TPP3 pairs
tpp3_pairs = (vdjdb_df['task'] == 'TPP3').sum()

# Print the results
print(f"\n**TPP Analysis for VDJdb with Concatenated Beta Data**")
print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
print(f"  - Gesamt Test-Paare: {len(vdjdb_df)}")

In [ ]:
# Negative Daten aus Train/Validation
negative_in_trainval = trainval_df[trainval_df['Binding'] == 0][["tcr_key", "Epitope"]]
negative_in_test = df_test[df_test['Binding'] == 0][["tcr_key", "Epitope"]]

# Prüfen, ob VDJdb-TCRs in den negativen Daten vorkommen
tpp3_tcr_overlap = vdjdb_df[~vdjdb_df['tcr_key'].isin(seen_tcrs) & vdjdb_df['tcr_key'].isin(set(negative_in_trainval['tcr_key']))]
tpp3_tcr_overlap_test = vdjdb_df[~vdjdb_df['tcr_key'].isin(seen_tcrs) & vdjdb_df['tcr_key'].isin(set(negative_in_test['tcr_key']))]

# Prüfen, ob VDJdb-Epitope in den negativen Daten vorkommen
tpp3_epitope_overlap = vdjdb_df[~vdjdb_df['Epitope'].isin(seen_epitopes) & vdjdb_df['Epitope'].isin(set(negative_in_trainval['Epitope']))]
tpp3_epitope_overlap_test = vdjdb_df[~vdjdb_df['Epitope'].isin(seen_epitopes) & vdjdb_df['Epitope'].isin(set(negative_in_trainval['Epitope']))]

print(f"VDJdb TPP3-Paare, deren TCR jetzt in negativen Daten aus Train/Validation vorkommt: {len(tpp3_tcr_overlap)}")
print(f"VDJdb TPP3-Paare, deren Epitope jetzt in negativen Daten aus Train/Validation vorkommt: {len(tpp3_epitope_overlap)}")

In [ ]:
# Load VDJdb test data
vdjdb_df = pd.read_csv(vdjdb_test_file, sep='\t') #f'{pipeline_data_cleaned}/VDJdb/VDJdb_cleaned_data_beta.tsv'
vdjdb_df["tcr_key"] = vdjdb_df["TRB_CDR3"].astype(str).str.strip()
vdjdb_df["Epitope"] = vdjdb_df["Epitope"].astype(str).str.strip()
vdjdb_df['task'] = vdjdb_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=False), axis=1)
# TPP3-Paare aus dem vdjdb File
original_tpp3 = vdjdb_df[vdjdb_df['task'] == 'TPP3'][["tcr_key", "Epitope"]]
# Prüfen, ob diese TPP3-Paare im finalen Testset noch vorhanden sind
tpp3_in_test = original_tpp3.merge(df_test, on=["tcr_key", "Epitope"], how="inner")
print(f"TPP3-Paare von cleaned data, die noch im finalen Testset vorhanden sind: {len(tpp3_in_test)}")

# Prüfen, welche TPP-Klasse die TPP3-Paare jetzt im finalen Testset haben
tpp3_in_test_with_task = original_tpp3.merge(df_test, on=["tcr_key", "Epitope"], how="inner")

# Zählen, wie viele der ursprünglichen TPP3-Paare jetzt welcher TPP-Klasse zugeordnet wurden
task_distribution = tpp3_in_test_with_task['task'].value_counts()

print(f"\n✅ TPP3-Paare von cleaned data, die noch im finalen Testset vorhanden sind: {len(tpp3_in_test_with_task)}")
print(f"🔄 Aktuelle TPP-Klassen dieser Paare im finalen Testset:")
print(task_distribution)

In [ ]:
# Prüfen, ob TCRs in negativen Daten des Train/Validation-Sets vorkommen
tcrs_in_negative_trainval = original_tpp3[original_tpp3['tcr_key'].isin(trainval_df[trainval_df['Binding'] == 0]['tcr_key'])]

# Prüfen, ob Epitope in negativen Daten des Train/Validation-Sets vorkommen
epitopes_in_negative_trainval = original_tpp3[original_tpp3['Epitope'].isin(trainval_df[trainval_df['Binding'] == 0]['Epitope'])]

print(f"⚠️ Anzahl TPP3-Paare, deren TCRs in den negativen Train/Validation-Daten vorkommen: {len(tcrs_in_negative_trainval)}")
print(f"⚠️ Anzahl TPP3-Paare, deren Epitope in den negativen Train/Validation-Daten vorkommen: {len(epitopes_in_negative_trainval)}")

# Prüfen, ob TCRs in den positiven Daten vorkommen
tcrs_in_positive_trainval = original_tpp3[original_tpp3['tcr_key'].isin(trainval_df[trainval_df['Binding'] == 1]['tcr_key'])]

# Prüfen, ob Epitope in den positiven Daten vorkommen
epitopes_in_positive_trainval = original_tpp3[original_tpp3['Epitope'].isin(trainval_df[trainval_df['Binding'] == 1]['Epitope'])]

print(f"✅ Anzahl TPP3-Paare, deren TCRs in den positiven Train/Validation-Daten vorkommen: {len(tcrs_in_positive_trainval)}")
print(f"✅ Anzahl TPP3-Paare, deren Epitope in den positiven Train/Validation-Daten vorkommen: {len(epitopes_in_positive_trainval)}")


In [2]:
pipeline_data = '../../../../data'
pipeline_data_plain = f'{pipeline_data}/plain_datasets'
pipeline_data_cleaned = f'{pipeline_data}/cleaned_datasets'
pipeline_data_concatenated = f'{pipeline_data}/concatenated_datasets'
pipeline_data_splitted = f'{pipeline_data}/splitted_datasets'
pipeline_data_temp_bucket = f'{pipeline_data}/temp'
precision = "allele"

import pandas as pd

# Datei-Pfade (ersetze `pipeline_data_splitted` und `precision` mit deinen Variablen)
train_file = f'{pipeline_data_splitted}/{precision}/beta/train.tsv'
validation_file = f'{pipeline_data_splitted}/{precision}/beta/validation.tsv'
test_file = f'{pipeline_data_splitted}/{precision}/beta/test.tsv'

# Helper-Funktion
def count_unique_epitopes(file_path, split_name):
    df = pd.read_csv(file_path, sep='\t')
    df['split'] = split_name
    return df.groupby('Binding')['Epitope'].nunique().reset_index(name='unique_epitopes').assign(split=split_name)

# Ergebnisse sammeln
results = pd.concat([
    count_unique_epitopes(train_file, 'train'),
    count_unique_epitopes(validation_file, 'val'),
    count_unique_epitopes(test_file, 'test')
])

# Umstrukturieren für bessere Übersicht
summary = results.pivot(index='split', columns='Binding', values='unique_epitopes').rename(columns={0: 'non-binding', 1: 'binding'}).fillna(0).astype(int)

print(summary)


Binding  non-binding  binding
split                        
test             724      293
train            976      981
val              802     1450


In [4]:
# Definierte Pfade für alle vier Kategorien
datasets = {
    "beta_allele": {
        "train": f'{pipeline_data_splitted}/{precision}/beta/train.tsv',
        "test": f'{pipeline_data_splitted}/{precision}/beta/test.tsv',
        "validation": f'{pipeline_data_splitted}/{precision}/beta/validation.tsv'
    }
}

In [6]:
# Zusätzliche Analyse für einzigartige Epitope
unique_epitopes = {}

for dataset_name, paths in datasets.items():
    train_df = pd.read_csv(paths["train"], sep='\t')
    test_df = pd.read_csv(paths["test"], sep='\t')
    validation_df = pd.read_csv(paths["validation"], sep='\t')
    
    # Extrahieren einzigartiger Epitope
    train_unique = set(train_df["Epitope"].unique())
    test_unique = set(test_df["Epitope"].unique())
    validation_unique = set(validation_df["Epitope"].unique())
    
    # Gesamtanzahl einzigartiger Epitope
    all_unique = train_unique | test_unique | validation_unique  # Vereinigung der Sets

    # Ergebnisse speichern
    unique_epitopes[dataset_name] = {
        "Train_Unique": len(train_unique),
        "Test_Unique": len(test_unique),
        "Validation_Unique": len(validation_unique),
        "Total_Unique": len(all_unique),
        "Train_Epitopes": train_unique,
        "Test_Epitopes": test_unique,
        "Validation_Epitopes": validation_unique
    }

# Ergebnisse anzeigen
for dataset, epitopes in unique_epitopes.items():
    print(f'--- {dataset.replace("_", " ").title()} ---')
    print(f'Anzahl einzigartiger Epitope im Trainingsdatensatz: {epitopes["Train_Unique"]}')
    print(f'Anzahl einzigartiger Epitope im Testdatensatz: {epitopes["Test_Unique"]}')
    print(f'Anzahl einzigartiger Epitope im Validierungsdatensatz: {epitopes["Validation_Unique"]}')
    print(f'Gesamtanzahl einzigartiger Epitope (Train + Test + Validation): {epitopes["Total_Unique"]}\n')


/tmp/ipykernel_417003/3828324703.py:5: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(paths["train"], sep='\t')


--- Beta Allele ---
Anzahl einzigartiger Epitope im Trainingsdatensatz: 987
Anzahl einzigartiger Epitope im Testdatensatz: 299
Anzahl einzigartiger Epitope im Validierungsdatensatz: 1456
Gesamtanzahl einzigartiger Epitope (Train + Test + Validation): 1896



In [8]:
# Anzahl der einzigartigen Epitope pro Binding und pro Datei
unique_epitopes_count = {}

for dataset, paths in datasets.items():
    train_df = pd.read_csv(paths["train"], sep='\t')
    test_df = pd.read_csv(paths["test"], sep='\t')
    validation_df = pd.read_csv(paths["validation"], sep='\t')
    
    # Einzigartige Epitope für Binding=0 und Binding=1
    train_unique = {
        0: len(set(train_df[train_df['Binding'] == 0]['Epitope'])),
        1: len(set(train_df[train_df['Binding'] == 1]['Epitope']))
    }
    test_unique = {
        0: len(set(test_df[test_df['Binding'] == 0]['Epitope'])),
        1: len(set(test_df[test_df['Binding'] == 1]['Epitope']))
    }
    validation_unique = {
        0: len(set(validation_df[validation_df['Binding'] == 0]['Epitope'])),
        1: len(set(validation_df[validation_df['Binding'] == 1]['Epitope']))
    }
    
    # Ergebnisse speichern
    unique_epitopes_count[dataset] = {
        "Train_Binding_0": train_unique[0],
        "Train_Binding_1": train_unique[1],
        "Test_Binding_0": test_unique[0],
        "Test_Binding_1": test_unique[1],
        "Validation_Binding_0": validation_unique[0],
        "Validation_Binding_1": validation_unique[1]
    }

# Ergebnisse ausgeben
for dataset, counts in unique_epitopes_count.items():
    print(f'--- {dataset.replace("_", " ").title()} Unique Epitope Counts ---')
    print(f'Train (Binding=0): {counts["Train_Binding_0"]}, Train (Binding=1): {counts["Train_Binding_1"]}')
    print(f'Test (Binding=0): {counts["Test_Binding_0"]}, Test (Binding=1): {counts["Test_Binding_1"]}')
    print(f'Validation (Binding=0): {counts["Validation_Binding_0"]}, Validation (Binding=1): {counts["Validation_Binding_1"]}\n')


--- Beta Allele Unique Epitope Counts ---
Train (Binding=0): 976, Train (Binding=1): 981
Test (Binding=0): 724, Test (Binding=1): 293
Validation (Binding=0): 810, Validation (Binding=1): 1450



In [10]:
import pandas as pd

# Dateien einlesen
train_df = pd.read_csv(train_file, sep='\t')
val_df = pd.read_csv(validation_file, sep='\t')
test_df = pd.read_csv(test_file, sep='\t')

# Funktion zur Ausgabe der Verteilung
def show_binding_distribution(df, name):
    counts = df['Binding'].value_counts().sort_index()
    total = counts.sum()
    print(f'--- {name} ---')
    for b in counts.index:
        percentage = (counts[b] / total) * 100
        print(f'Binding = {b}: {counts[b]} Beispiele ({percentage:.2f}%)')
    print(f'Total: {total} Beispiele\n')

# Ausgabe
show_binding_distribution(train_df, 'Train')
show_binding_distribution(val_df, 'Validation')
show_binding_distribution(test_df, 'Test')

--- Train ---
Binding = 0: 133876 Beispiele (48.86%)
Binding = 1: 140126 Beispiele (51.14%)
Total: 274002 Beispiele

--- Validation ---
Binding = 0: 175160 Beispiele (83.33%)
Binding = 1: 35032 Beispiele (16.67%)
Total: 210192 Beispiele

--- Test ---
Binding = 0: 44445 Beispiele (83.32%)
Binding = 1: 8900 Beispiele (16.68%)
Total: 53345 Beispiele



In [12]:
import zipfile

zip_path = f"{pipeline_data_plain}/data.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("📦 Inhalt von data.zip:")
    zip_ref.printdir()


📦 Inhalt von data.zip:
File Name                                             Modified             Size
data/                                          2024-12-20 00:12:38            0
data/cleaned_datasets/                         2024-12-20 00:11:34            0
data/cleaned_datasets/IEDB/                    2024-12-20 00:11:32            0
data/cleaned_datasets/IEDB/IEDB_cleaned_data_beta.csv 2024-12-20 00:11:32     13347360
data/cleaned_datasets/IEDB/IEDB_cleaned_data_paired.csv 2024-12-20 00:11:32      2009011
data/cleaned_datasets/McPas/                   2024-12-20 00:11:36            0
data/cleaned_datasets/McPas/McPAS_cleaned_data_beta.tsv 2024-12-20 00:11:36       752763
data/cleaned_datasets/McPas/McPAS_cleaned_data_paired.tsv 2024-12-20 00:11:36       185984
data/cleaned_datasets/VDJdb/                   2024-12-20 00:11:34            0
data/cleaned_datasets/VDJdb/VDJdb_cleaned_data_beta.tsv 2024-12-20 00:11:34      3850836
data/cleaned_datasets/VDJdb/VDJdb_cleaned_data_paire

In [11]:
import zipfile
import os
import shutil

# Pfade und Zielverzeichnis definieren
zip_path = f"{pipeline_data_plain}/data.zip"
output_dir = f"{pipeline_data}/ba_splitted"  # Zielordner

# Die drei gewünschten Dateien im ZIP
target_files = [
    "data/splitted_datasets/allele/beta/train.tsv",
    "data/splitted_datasets/allele/beta/validation.tsv",
    "data/splitted_datasets/allele/beta/test.tsv"
]

# Zielverzeichnis erstellen, falls nicht vorhanden
os.makedirs(output_dir, exist_ok=True)

# Extraktion
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in target_files:
        # Temporär an ursprünglichem Pfad extrahieren
        extracted_path = zip_ref.extract(file)

        # Nur den Dateinamen extrahieren (z.B. "train.tsv")
        filename = os.path.basename(file)

        # Zielpfad festlegen und Datei verschieben
        final_path = os.path.join(output_dir, filename)
        shutil.move(extracted_path, final_path)

print(f"✅ Dateien erfolgreich extrahiert nach: {output_dir}")

✅ Dateien erfolgreich extrahiert nach: ../../../../data/ba_splitted
